In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Data Imports from src/sqldb.py
# DO NOT CHANGE BEFORE CHANGING SOURCE FILE
import os
import pandas as pd
import sqlite3

class SQLDatabase:
    """API for interacting with SQL database."""
    
    def __init__(self, db_name=None):
        if db_name is None:
            # CHANGING DEFINITION SPECIFICALLY FOR JUPYTER NOTEBOOK
            # _path_to_db = os.path.abspath("dbs/links.db")
            _path_to_db = os.path.join(
                os.path.dirname(os.getcwd()),
                "dbs",
                "links.db"
            )
        else:
            _path_to_db = db_name
        try:
            self._connection = sqlite3.connect(_path_to_db)
            self._cursor = self._connection.cursor()
        except:
            raise ValueError("Bad Connection")

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.close()

    def close(self):
        self._connection.close()

    def union_events(self, data, table):
        for item in data:
            self.add_new(item, table)
        self._connection.commit()

    def add_new(self, item, table):
        if table == "event":
            try:
                self._cursor.execute(
                    """
                    INSERT INTO event (name, link, date)
                    VALUES (?, ?, ?)
                    """, item
                )
            except sqlite3.IntegrityError:
                pass

        elif table == "pilot":
            if len(item) == 1:
                item += [""]
            try:
                self._cursor.execute(
                    """
                    INSERT INTO pilot (firstName, lastName)
                    values (?, ?)
                    """, item
                )
            except sqlite3.IntegrityError:
                pass

        elif table == "deck":
            query = """
            INSERT INTO deck (eventId, pilotId, deckUrl, name, rank)
            VALUES (?, ?, ?, ?, ?)
            """
            event_id = self._cursor.execute("""
                SELECT id
                FROM event
                WHERE link = ?
                """, (item[0], )
            ).fetchone()[0]
            pilot_id = self._cursor.execute("""
                SELECT id
                FROM pilot
                WHERE (firstName || lastName) = ?
                """, ("".join(item[1].split(maxsplit=1)),)
            ).fetchone()[0]
            items_to_insert = item[2:]
            try:
                self._cursor.execute(query, (event_id, pilot_id) + items_to_insert)
            except sqlite3.IntegrityError:
                pass

        elif table == "decklist":
            query = """
            INSERT INTO deckList (cardId, deckId, count, slot)
            VALUES (?, ?, ?, ?)
            """
            try:
                self._cursor.execute(query, item)
            except sqlite3.IntegrityError:
                pass

        elif table == "card":
            query = """
            INSERT INTO card (setNumber, setName, name, cmc, color, standardLegality, oracle_text, mana_cost)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """
            try:
                self._cursor.execute(query, item)
            except sqlite3.IntegrityError:
                pass

    def get_dataframe_from(self, table):
        query = f"SELECT * FROM {table}"
        return pd.read_sql(query, self._connection)


In [3]:
# Data Objects
with SQLDatabase() as sql_db:
    event_table = sql_db.get_dataframe_from("event")
    deck_table = sql_db.get_dataframe_from("deck")
    deck_list_table = sql_db.get_dataframe_from("deckList")
    card_table = sql_db.get_dataframe_from("card")
    pilot_table = sql_db.get_dataframe_from("pilot")

In [4]:
# Clean Dates
event_table["date"] = pd.to_datetime(event_table["date"], dayfirst=True)

# Renaming ID Columns
event_table.rename(columns={"id": "eventId"}, inplace=True)
deck_table.rename(columns={"id": "deckId"}, inplace=True)
pilot_table.rename(columns={"id": "pilotId"}, inplace=True)

# Adding easier ID column to card_table
card_table["setNumber"] = card_table["setNumber"].str.zfill(3)
card_table["cardId"] = card_table["setNumber"] + card_table["setName"]

In [5]:
# Adding archetypes to deck_table
archetypes = {
    "aggro": ["aggro", "red deck wins", "weenie"],
    "control": ["control", "doom", "4cc", "yorion"],
    "mill": ["mill"],
    "ramp": ["ramp", "omnath"],
    "rogue": ["rogue"],
    "flash": ["flash"],
    "adventure": ["adventure"],
    "food": ["food"],
    "midrange": ["midrange"]
}

for archetype in archetypes:
    for placeholder in archetypes[archetype]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "archetype"
        ] = archetype

In [6]:
# Adding easy color groups to deck_table
categories = {
    "mono white": ["mono white", "mono-white", "white weenie", "weenie white"],
    "mono blue": ["mono blue", "mono-blue"],
    "mono black": ["mono black", "mono-black"],
    "mono red": ["mono red", "mono-red", "red deck wins"],
    "mono green": ["mono green", "mono-green", "monogreen"],
    "azorius": ["azorius", "uw", "wu"],
    "orzhov": ["orzhov", "wb", "bw"],
    "boros": ["boros", "wr", "rw", "winota"],
    "selesnya": ["selesnya", "wg", "gw"],
    "dimir": ["dimir", "ub"],
    "izzet": ["izzet", "ur"],
    "simic": ["simic", "gu"],
    "rakdos": ["rakdos", "br", "rb"],
    "golgari": ["golgari", "bg", "gb"],
    "gruul": ["gruul", "rg"],
    "bant": ["bant", "gwu", "guw", "ugw", "uwg", "wgu", "wug"],
    "esper": ["esper", "wub"],
    "grixis": ["grixis"],
    "jund": ["jund"],
    "naya": ["naya"],
    "jeskai": ["jeskai"],
    "sultai": ["sultai"],
    "mardu": ["mardu", "kroxa doom"],
    "temur": ["temur"],
    "abzan": ["abzan"],
    "4-color": ["omnath", "4c"],
    "colorless": ["forsaken monument"]
}

for category in categories:
    for placeholder in categories[category]:
        deck_table.loc[
            (deck_table.name.str.lower().str.contains(placeholder)),
            "category"
        ] = category

deck_table.loc[
    (deck_table.name.str.lower() == "gyruda"),
    "category"
] = "4-color"

In [7]:
# Replace all the abu codes with updated codes for basic lands 
# and Disenchant & Unsummon
abu_codes = {
    "092abu": "266eld",
    "170abu": "262eld",
    "130abu": "254eld",
    "196abu": "250eld",
    "249abu": "258eld",
    "065abu": "010znr",
    "271abu": "078m20"
}

other_codes = {
    "171cmd": "204m21", # Scavenging Ooze
    "356eld": "", # Rankle, Master of Pranks
    "367eld": "", # Torbran, Thane of Red Fell
    "335eld": "", # Charming Prince
    "346eld": "",
    
}

card_table["cardId"].replace(to_replace=abu_codes, inplace=True)
deck_list_table["cardId"].replace(to_replace=abu_codes, inplace=True)

In [15]:
card_table.loc[
    card_table.cardId == "054m12"
]

,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text,cardId
3425,054,m12,Frost Breath,3,U,{2}{U},legal,Tap up to two target creatures. Those creature...,054m12


In [8]:
deck_list_table.loc[
    ~(deck_list_table.cardId.isin(card_table.cardId))
]["cardId"].value_counts()

171cmd     3303
083urs     1260
356eld      631
364znr      486
328iko      423
367eld      407
1136th      391
335eld      368
036str      284
05110m      276
107dar      267
346eld      142
163gui      116
162gui      104
23215m       81
237urs       64
00514c       56
00410m       48
144gui       36
1276th       28
0496th       20
021sha       12
309thb        8
001unk        4
054m12        4
079sta        4
05412m        4
003ivg        4
10113m        2
12410m        1
Name: cardId, dtype: int64

In [16]:
events_and_decks = pd.merge(
    event_table, 
    deck_table, 
    on="eventId",
    suffixes=["_event", "_deck"]
).drop_duplicates()

events_decks_pilots = pd.merge(
    events_and_decks,
    pilot_table,
    on="pilotId",
    suffixes=[None, "_pilot"]
).drop_duplicates()

full_table_no_cards = pd.merge(
    events_decks_pilots,
    deck_list_table,
    on="deckId",
    suffixes=[None, "_decklist"]
).drop_duplicates()

full_table = pd.merge(
    full_table_no_cards,
    card_table,
    on="cardId",
    suffixes=[None, "_cards"]
)[[
    "eventId",
    "name_event",
    "date",
    "deckId",
    "pilotId",
    "name_deck",
    "firstName",
    "lastName",
    "cardId",
    "name",
    "count",
    "color",
    "slot",
    "archetype",
    "category"
]].drop_duplicates().sort_values(by=["eventId", "deckId", "slot"]).reset_index(drop=True)

In [17]:
full_table.loc[
    full_table["date"] >= "2020-09-25"
].groupby("archetype")["deckId"].count().sort_values(ascending=False)

archetype
aggro        18028
control      11600
ramp          5264
mill          3020
rogue         1887
adventure     1871
midrange       113
food            86
flash           26
Name: deckId, dtype: int64

In [18]:
full_table.loc[
    full_table["date"] < "2020-09-28"
].groupby("category")["deckId"].count().sort_values(ascending=False)

category
4-color       3644
sultai        2898
izzet         1122
mono green     745
mono red       596
temur          545
gruul          543
dimir          360
rakdos         341
boros          262
azorius        131
mono black     102
simic          101
esper           94
orzhov          81
selesnya        62
jund            28
mardu           25
golgari         23
Name: deckId, dtype: int64

In [19]:
full_table.loc[
    full_table["deckId"] == 1063
]

,eventId,name_event,date,deckId,pilotId,name_deck,firstName,lastName,cardId,name,count,color,slot,archetype,category
27765,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,266eld,Forest,2,G,md,NaN,4-color
27766,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,262eld,Mountain,2,R,md,NaN,4-color
27767,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,254eld,Island,2,U,md,NaN,4-color
27768,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,250eld,Plains,1,W,md,NaN,4-color
27769,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,168m11,Cultivate,4,G,md,NaN,4-color
27770,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,164iko,Migration Path,4,G,md,NaN,4-color
27771,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,164m21,Terror of the Peaks,4,R,md,NaN,4-color
27772,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,220iko,Fiend Artisan,4,BG,md,NaN,4-color
27773,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,160eld,Gilded Goose,4,G,md,NaN,4-color
27774,220,MTGO Standard Challenge,2020-09-07,1063,797,Gyruda,Synergizer,,071thb,"Thassa, Deep-Dwelling",4,U,md,NaN,4-color


In [406]:
full_table.loc[
    full_table.category.isna()
][["deckId", "name_deck"]]

,deckId,name_deck
9199,360,Other - Aggro
9200,360,Other - Aggro
9201,360,Other - Aggro
9202,360,Other - Aggro
9203,360,Other - Aggro
...,...,...
24489,934,
24490,934,
24491,934,
24492,934,


In [407]:
deck1 = pd.merge(
    events_decks_pilots_deck_lists.loc[events_decks_pilots_deck_lists["deckId"] == 1],
    card_table,
    on="cardId",
    suffixes=[None, "_card"]
)

In [13]:
full_table.loc[
    ~(full_table.name.isin(["Forest", "Swamp", "Mountain", "Island", "Plains"]))
    & (full_table["date"] >= "2020-11-15")
].groupby("name")["count"].sum().sort_values(ascending=False).head(30)

name
Fabled Passage                                            771
Lovestruck Beast // Heart's Desire                        580
Kazandu Mammoth // Kazandu Valley                         539
Bonecrusher Giant // Stomp                                517
Mystical Dispute                                          452
Cragcrown Pathway // Timbercrown Pathway                  428
Edgewall Innkeeper                                        388
The Great Henge                                           362
Shatterskull Smashing // Shatterskull, the Hammer Pass    356
Clearwater Pathway // Murkwater Pathway                   347
Heartless Act                                             302
Embercleave                                               289
Ox of Agonas                                              288
Negate                                                    284
Vivien, Monsters' Advocate                                273
Shark Typhoon                                             271
Tem

### QUESTION 1: 

How has color dominance changed over time?